In [2]:
import sys
sys.path.append('/home/sharedrive/nafi/traffic/lib/python3.9/site-packages')

import pandas as pd
from chronos import ChronosPipeline
import torch
from table_evaluator import TableEvaluator
import matplotlib.pyplot as plt
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import Metadata
import numpy as np
from ctgan import CTGAN
from ctgan import load_demo
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
from table_evaluator import TableEvaluator
import pickle
import os

In [3]:
cic_train_df = pd.read_csv("./datasets/CICIDS_clean_train.csv")
cic_test_df = pd.read_csv("./datasets/CICIDS_clean_test.csv")
cic_train_df.shape, cic_test_df.shape

((1979513, 25), (848363, 25))

In [3]:
label_encoder = LabelEncoder()
scaler = StandardScaler()

In [4]:
classes = ['Mal_Mod2', 'Mal_Del', 'Mal_Mod', 'Mal_Estab', 'Normal']

def train(train_df, epochi, file_name, reduced_test_df=cic_test_df):
    for num_sample in [1000]:
        print(f"{file_name} epoch={epochi} samples={num_sample}")

        # divide to x_train and y_train from reduced_train_df
        x_train = train_df.drop(columns=["Label"])
        y_train = train_df["Label"]
        x_test = reduced_test_df.drop(columns=["Label"])
        y_test = reduced_test_df["Label"]
        
        y_train = label_encoder.fit_transform(y_train) # Encode string labels into numeric format
        y_test = label_encoder.transform(y_test)
        x_train = scaler.fit_transform(x_train) # Standardize the features
        x_test = scaler.transform(x_test)

        RF_model = RandomForestClassifier(random_state=1002, verbose=2)
        RF_model.fit(x_train, y_train) #For sklearn no one hot encoding

        pred_rf = RF_model.predict(x_test)


        pickle.dump(RF_model, open(f'./models/{file_name}sample{num_sample}epoch{epochi}.pkl', 'wb'))

        print(f"Classification Report for Sample Size {num_sample}:")
        print(classification_report(y_test, pred_rf, digits=4, target_names=label_encoder.classes_))

In [6]:
train(train_df=cic_train_df, epochi=1500, file_name='real_from_cicids')

real_from_cicids epoch=1500 samples=1000
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  3.4min


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    5.1s


Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN       1.00      1.00      1.00    681396
                      Bot       0.62      0.76      0.68       587
                     DDoS       1.00      1.00      1.00     38408
            DoS_GoldenEye       1.00      0.99      0.99      3088
                 DoS_Hulk       1.00      1.00      1.00     69037
         DoS_Slowhttptest       0.95      0.99      0.97      1650
            DoS_slowloris       1.00      1.00      1.00      1739
              FTP-Patator       1.00      1.00      1.00      2380
               Heartbleed       1.00      0.67      0.80         3
             Infiltration       1.00      0.45      0.62        11
                 PortScan       0.99      1.00      1.00     47641
              SSH-Patator       1.00      0.99      0.99      1769
  Web_Attack__Brute_Force       0.71      0.77      0.74       452
Web_Attack__Sql_I